In [17]:
# mac475의 ipython 표준 style을 적용함
from IPython.core.display import HTML
styles = open("../styles/custom.css", "r").read()
HTML( styles )

    0. Dataset 읽기

    1. Feature 추출 function

    2. Label Encoding function

    3. Label Encoding용 Dataset
    
        3.1 train/ test dataset merge

    4. Modeling

        4.1 Merged features중, modeling시 제거할 feature 관리

        4.2 Train : 연/ 월/ 일 Feature 추출

        4.3 Train : Tube와 merge통한 Feature 확장

        4.4 Train : Merged dataset Label Encoding 수행

        4.5 Train : 불필요 Feature 제거

        4.6 Test : Dataset 읽기

        4.7 Test : 연/ 월/ 일 Feature 추출

        4.8 Test : Tube와 merge통한 Feature 확장

        4.9 Test : Merged dataset Label Encoding 수행

        4.10 Test : 불필요 Featrue 제거

#0. Mode 설정

In [18]:
mode = True
# mode = False

#0. Dataset 읽기

In [19]:
import pandas as pd
import numpy as np

df_train = pd.read_csv( './dataset/train_set.csv' )    # data를 읽어들인다.
df_test = pd.read_csv( './dataset/test_set.csv' )    # data를 읽어들인다.

#1. Feature 추출 function
* quote_date 활용하여 year, month, day 확보

In [20]:
import datetime

def extract_year_month_day_from_quote_date( p_df ) :
    p_df[ 'quote_date' ] = pd.to_datetime( p_df[ 'quote_date' ] )    # string을 datetime으로 형변환
    p_df[ 'year' ] = p_df[ 'quote_date' ].dt.year    # 연도
    p_df[ 'month' ] = p_df[ 'quote_date' ].dt.month    # 월
    p_df[ 'day' ] = p_df[ 'quote_date' ].dt.day    # 일

    return p_df

#2. Label Encoding function
* 현재, train/ test dataset과 tube dataset과 merge하는 case까지 반영하여 encoder 관리

In [21]:
from sklearn import preprocessing

# le_tube_assembly_id = preprocessing.LabelEncoder()
le_supplier = preprocessing.LabelEncoder()
le_bracket_pricing = preprocessing.LabelEncoder()

le_material_id = preprocessing.LabelEncoder()
le_end_a_1x = preprocessing.LabelEncoder()
le_end_a_2x = preprocessing.LabelEncoder()
le_end_x_1x = preprocessing.LabelEncoder()
le_end_x_2x = preprocessing.LabelEncoder()
le_end_a = preprocessing.LabelEncoder()
le_end_x = preprocessing.LabelEncoder()

le_component_id = preprocessing.LabelEncoder()    # component의 경우, master dataset이 별도로 존재하므로
df_components = pd.read_csv( './dataset/02.ml.verified.dataset/components.verified.csv' )
le_component_id.fit( df_components[ 'component_id' ] )
df_components = None

le_spec_id = preprocessing.LabelEncoder()    # spec의 경우, spec meta dataset을 별도로 생성했음
df_spec_meta = pd.read_csv( './dataset/01.original.dataset/spec_meta.csv' )
le_spec_id.fit( df_spec_meta[ 'spec' ].append( pd.Series( [ '-1' ] ) ) )
df_spec_meta = ''

def executeLabelEncoding( p_df, is_init ) :
    if is_init == True :    # training dataset인 경우, label encoder 생성 및 fitting 수행
        p_df[ 'bracket_pricing' ] = le_bracket_pricing.fit_transform( p_df[ 'bracket_pricing' ] )

        p_df[ 'supplier' ] = le_supplier.fit_transform( p_df[ 'supplier' ] )
        p_df[ 'material_id' ] = le_material_id.fit_transform( p_df[ 'material_id' ] )
        
        p_df[ 'end_a_1x' ] = le_end_a_1x.fit_transform( p_df[ 'end_a_1x' ] )
        p_df[ 'end_a_2x' ] = le_end_a_2x.fit_transform( p_df[ 'end_a_2x' ] )
        p_df[ 'end_x_1x' ] = le_end_x_1x.fit_transform( p_df[ 'end_x_1x' ] )
        p_df[ 'end_x_2x' ] = le_end_x_2x.fit_transform( p_df[ 'end_x_2x' ] )
        p_df[ 'end_a' ] = le_end_a.fit_transform( p_df[ 'end_a' ] )
        p_df[ 'end_x' ] = le_end_x.fit_transform( p_df[ 'end_x' ] )
    else :
        p_df[ 'bracket_pricing' ] = le_bracket_pricing.transform( p_df[ 'bracket_pricing' ] )

        p_df[ 'supplier' ] = le_supplier.transform( p_df[ 'supplier' ] )
        p_df[ 'material_id' ] = le_material_id.transform( p_df[ 'material_id' ] )
        
        p_df[ 'end_a_1x' ] = le_end_a_1x.transform( p_df[ 'end_a_1x' ] )
        p_df[ 'end_a_2x' ] = le_end_a_2x.transform( p_df[ 'end_a_2x' ] )
        p_df[ 'end_x_1x' ] = le_end_x_1x.transform( p_df[ 'end_x_1x' ] )
        p_df[ 'end_x_2x' ] = le_end_x_2x.transform( p_df[ 'end_x_2x' ] )
        p_df[ 'end_a' ] = le_end_a.transform( p_df[ 'end_a' ] )
        p_df[ 'end_x' ] = le_end_x.transform( p_df[ 'end_x' ] )
    
    # 무조건 수행
    for i in range( 1, 9 ) :    # bill_of_materials에서 merge된 component_id_1~8을 label encoding 수행
        comp_str = 'component_id_' + str( i )
        p_df[ comp_str ] = le_component_id.transform( p_df[ comp_str ] )
        
    for i in range( 1, 11 ) :    # specs에서 merge된 spec1~10을 label encoding 수행
        spec_str = 'spec' + str( i )
        p_df[ spec_str ] = le_spec_id.transform( p_df[ spec_str ] )        
    
    return p_df

.

#3. Label Encoding용 Dataset
* categorical data를 모두 포함해야 하므로, tube join후 기반으로 label encoder를 fitting한다
* label encoding의 대상은 bracket_pricing, supplier, material_id, end_a_1x 시리즈, end_a 시리즈 feature이다

##3.1 train/ test dataset merge

In [22]:
df_train[ 'id' ] = '99999'    # test와 join위해 feature 추가 : 99999는 train dataset이다

In [23]:
df_merged = df_train.append( df_test )    # train과 test df를 merge한다.

In [24]:
df_merged.tail( 3 )

,annual_usage,bracket_pricing,cost,id,min_order_quantity,quantity,quote_date,supplier,tube_assembly_id
30232,883,No,NaN,30233,85,85,2013-09-10,S-0041,TA-21194
30233,28,No,NaN,30234,1,1,2003-09-16,S-0026,TA-21198
30234,28,No,NaN,30235,1,1,2003-09-16,S-0026,TA-21199


In [25]:
df_merged.count()

annual_usage          60448
bracket_pricing       60448
cost                  30213
id                    60448
min_order_quantity    60448
quantity              60448
quote_date            60448
supplier              60448
tube_assembly_id      60448
dtype: int64

##3.1 tube_bill_specs dataset merge 및 label encoding

In [26]:
# df_tube = pd.read_csv( './dataset/tube.material_id.verified.csv' )
# df_tube_bill = pd.read_csv( './dataset/tube_bill_merged.csv', dtype = { 'component_id_8' : str } )
# df_tube_bill_specs = pd.read_csv( './dataset/tube_bill_specs_merged.csv',
#                                   dtype = { 'component_id_8' : str, 'spec9' : str, 'spec10' : str } )
# df_tube_bill_specs_end = pd.read_csv( './dataset/tube_bill_specs_end_merged.csv',
#                                   dtype = { 'component_id_8' : str, 'spec9' : str, 'spec10' : str } )
df_tube_bill_specs_end = pd.read_csv( './dataset/03.merged/tube_full_tmp.csv',
                                  dtype = { 'component_id_8' : str, 'spec9' : str, 'spec10' : str,
                                            'component_id_7' : str, 'spec8' : str, '69' : str } )
# df_tube_bill_specs.columns[30], df_tube_bill_specs.columns[43], df_tube_bill_specs.columns[44]

C:\Anaconda3\lib\site-packages\pandas\io\parsers.py:1159: DtypeWarning: Columns (26,38,39,40,41,102,211) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [27]:
# df_tube_bill_specs_end.columns[28], df_tube_bill_specs_end.columns[42], df_tube_bill_specs_end.columns[101]

In [28]:
df_tube_bill_specs_end.head( 3 )

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,...,128_y,129_y,130_y,131_y,134_y,135_y,136_y,138_y,34_y.4,140_y
0,TA-00001,SP-0035,12.70,1.65,164,5,38.10,N,N,N,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,TA-00002,SP-0019,6.35,0.71,137,8,19.05,N,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00003,SP-0019,6.35,0.71,127,7,19.05,N,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# df_merged = df_merged.merge( df_tube, how = 'inner', on = 'tube_assembly_id' )
df_merged = df_merged.merge( df_tube_bill_specs_end, how = 'inner', on = 'tube_assembly_id' )

In [30]:
df_merged.count()

annual_usage          60448
bracket_pricing       60448
cost                  30213
id                    60448
min_order_quantity    60448
quantity              60448
quote_date            60448
supplier              60448
tube_assembly_id      60448
material_id           60448
diameter              60448
wall                  60448
length                60448
num_bends             60448
bend_radius           60448
...
119_y     42149
122_y     42149
123_y     42149
124_y     42149
125_y     42149
128_y     42149
129_y     42149
130_y     42149
131_y     42149
134_y     42149
135_y     42149
136_y     42149
138_y     42149
34_y.4    42149
140_y     42149
Length: 274, dtype: int64

In [31]:
df_merged.columns    # merged feature를 확인

Index(['annual_usage', 'bracket_pricing', 'cost', 'id', 'min_order_quantity', 'quantity', 'quote_date', 'supplier', 'tube_assembly_id', 'material_id', 'diameter', 'wall', 'length', 'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other', 'component_id_1', 'quantity_1', 'component_id_2', 'quantity_2', 'component_id_3', 'quantity_3', 'component_id_4', 'quantity_4', 'component_id_5', 'quantity_5', 'component_id_6', 'quantity_6', 'component_id_7', 'quantity_7', 'component_id_8', 'quantity_8', 'comp_type_count', 'comp_total_count', 'tube_volume', 'spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6', 'spec7', 'spec8', 'spec9', 'spec10', 'spec_type_count', 'forming_x', 'forming_y', 'component_id_x', '4_x', '5_x', '8_x', '9_x', '10_x', '11_x', '14_x', '15_x', '16_x', '17_x', '18_x', '19_x', '20_x', '21_x', '23_x', '25_x', '26_x', '27_x', '28_x', '29_x', '30_x', '31_x', '32_x', '33_x', '34_x', '35_x', '37_x', '38_x', '39_x

In [32]:
def process_nulls( p_df ) :
    for i in range( 1, 9 ) :    # df_merged내의 null을 -1으로 채워둔다
        comp_str = 'component_id_' + str( i )
        quan_str = 'quantity_' + str( i )

        print( comp_str, quan_str )
        
        p_df[ comp_str ].fillna( '-1', inplace = True )
#         p_df[ quan_str ].fillna( -1, inplace = True )
        p_df[ quan_str ].fillna( 0, inplace = True )
    
    for i in range( 1, 11 ) :    # df_merged내의 null을 -1으로 채워둔다
        spec_str = 'spec' + str( i )

        print( spec_str )
        
        p_df[ spec_str ].fillna( '-1', inplace = True )

    p_df.fillna( -1, inplace = True )
        
    return p_df

In [33]:
df_merged = process_nulls( df_merged )    # merge된 df내의 null 처리

component_id_1 quantity_1
component_id_2 quantity_2
component_id_3 quantity_3
component_id_4 quantity_4
component_id_5 quantity_5
component_id_6 quantity_6
component_id_7 quantity_7
component_id_8 quantity_8
spec1
spec2
spec3
spec4
spec5
spec6
spec7
spec8
spec9
spec10


In [34]:
# df_merged[ 'component_id_1' ]

In [35]:
# df_merged[ 'component_id_2' ]

In [36]:
df_merged = executeLabelEncoding( df_merged, is_init = True )    # label encoding을 수행한다

In [37]:
df_merged.count()

annual_usage          60448
bracket_pricing       60448
cost                  60448
id                    60448
min_order_quantity    60448
quantity              60448
quote_date            60448
supplier              60448
tube_assembly_id      60448
material_id           60448
diameter              60448
wall                  60448
length                60448
num_bends             60448
bend_radius           60448
...
119_y     60448
122_y     60448
123_y     60448
124_y     60448
125_y     60448
128_y     60448
129_y     60448
130_y     60448
131_y     60448
134_y     60448
135_y     60448
136_y     60448
138_y     60448
34_y.4    60448
140_y     60448
Length: 274, dtype: int64

In [38]:
# df_merged.to_csv( './dataset/xxxxxxxxxxxxxxxxxx.verified.csv', index = False )
df_merged = ''

In [39]:
df_train.drop( 'id', axis = 1, inplace = True )    # 일단, 필요없는 것들 제거 : df_train을 원상 복구해둠
df_test = ''    # 일단, 필요없는 것들 제거
df_merged = ''    # 일단, 필요없는 것들 제거

#4. Modeling

##4.1 Merged features중, modeling시 제거할 feature 관리

In [40]:
list_for_remove = []    # 전역변수의 선언
def executeFeatureRemoval( p_df ) :
    # 제거하고자 하는 feature list
    global list_for_remove    # 전역변수 명시
    list_for_remove = [    # 82.7% 정확도 feature list : default
                        'tube_assembly_id',
#                         'supplier',

                        'quote_date',
#                         'annual_usage',
#                         'min_order_quantity',
        
    #                     'bracket_pricing',
    #                     'quantity',
        
#                         'year',
#                         'month',
#                         'day',

#                         'material_id',
#                         'diameter',
#                         'wall',
#                         'length',
    
#                         'num_bends',
#                         'bend_radius',
    
#                         'end_a_1x',
#                         'end_a_2x',
#                         'end_x_1x',
#                         'end_x_2x',
#                         'end_a',
#                         'end_x',
    
#                         'num_boss',
#                         'num_bracket',
#                         'other',
        
#                         'component_id_1',
#                         'quantity_1',
#                         'component_id_2',
#                         'quantity_2',
#                         'component_id_3',
#                         'quantity_3',
#                         'component_id_4',
#                         'quantity_4',
        
                        'component_id_5',
                        'quantity_5',
                        'component_id_6',
                        'quantity_6',
                        'component_id_7',
                        'quantity_7',
                        'component_id_8',
                        'quantity_8',
        
#                         'comp_type_count',        
#                         'comp_total_count',        
        
#                         'tube_volume',

#                         'spec1',
                        'spec2',
                        'spec3',
                        'spec4',
                        'spec5',
                        'spec6',
                        'spec7',
                        'spec8',
                        'spec9',
                        'spec10',
#                         'spec_type_count',

#                         'forming_x',
#                         'forming_y',
                        
                        'component_id_x', 'component_id_y', 
                        '69_x', '69_y',
                      ]      

    return p_df.drop( list_for_remove, axis = 1, inplace = False )

##4.2 Train : 연/ 월/ 일 Feature 추출

In [41]:
df_train = extract_year_month_day_from_quote_date( df_train )    # feature 처리를 수행

##4.3 Train : Tube와 merge통한 Feature 확장

In [42]:
# df_train_merged = df_train.merge( df_tube, how = 'inner', on = 'tube_assembly_id' )
# df_train_merged = df_train.merge( df_tube_bill, how = 'inner', on = 'tube_assembly_id' )
# df_train_merged = df_train.merge( df_tube_bill_specs, how = 'inner', on = 'tube_assembly_id' )
df_train_merged = df_train.merge( df_tube_bill_specs_end, how = 'inner', on = 'tube_assembly_id' )

##4.4 Train : Merged dataset Label Encoding 수행

In [43]:
df_train_merged = process_nulls( df_train_merged )
df_train_merged = executeLabelEncoding( df_train_merged, is_init = False )    # label encoding을 수행한다

component_id_1 quantity_1
component_id_2 quantity_2
component_id_3 quantity_3
component_id_4 quantity_4
component_id_5 quantity_5
component_id_6 quantity_6
component_id_7 quantity_7
component_id_8 quantity_8
spec1
spec2
spec3
spec4
spec5
spec6
spec7
spec8
spec9
spec10


##4.5 Train : 불필요 Feature 제거

In [44]:
df_train_merged = executeFeatureRemoval( df_train_merged )    # feature removal 처리를 수행
df_train_merged.head( 3 )

,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,cost,year,month,day,material_id,...,128_y,129_y,130_y,131_y,134_y,135_y,136_y,138_y,34_y.4,140_y
0,41,0,0,1,1,21.905933,2013,7,7,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,41,0,0,1,2,12.341214,2013,7,7,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,41,0,0,1,5,6.601826,2013,7,7,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


.

##4.6 Test : Dataset 읽기

In [45]:
df_test = pd.read_csv( './dataset/test_set.csv' )    # data를 읽어들인다.

##4.7 Test : 연/ 월/ 일 Feature 추출

In [46]:
df_test = extract_year_month_day_from_quote_date( df_test )    # feature 처리를 수행

df_result = pd.DataFrame( df_test[ 'id' ], columns = ['id'] )    # 결과용 dataframe을 생성
df_test.drop( 'id', axis = 1, inplace = True )    # id feature는 일단 제거

##4.8 Test : Tube와 merge통한 Feature 확장

In [47]:
# df_test_merged = df_test.merge( df_tube, how = 'inner', on = 'tube_assembly_id' )
# df_test_merged = df_test.merge( df_tube_bill, how = 'inner', on = 'tube_assembly_id' )
# df_test_merged = df_test.merge( df_tube_bill_specs, how = 'inner', on = 'tube_assembly_id' )
df_test_merged = df_test.merge( df_tube_bill_specs_end, how = 'inner', on = 'tube_assembly_id' )

In [48]:
df_test_merged.head( 3 )

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,year,month,day,...,128_y,129_y,130_y,131_y,134_y,135_y,136_y,138_y,34_y.4,140_y
0,TA-00001,S-0066,2013-06-23,0,0,Yes,1,2013,6,23,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,TA-00001,S-0066,2013-06-23,0,0,Yes,2,2013,6,23,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,TA-00001,S-0066,2013-06-23,0,0,Yes,5,2013,6,23,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


##4.9 Test : Merged dataset Label Encoding 수행

In [49]:
df_test_merged = process_nulls( df_test_merged )
df_test_merged = executeLabelEncoding( df_test_merged, is_init = False )    # label encoding을 수행한다

component_id_1 quantity_1
component_id_2 quantity_2
component_id_3 quantity_3
component_id_4 quantity_4
component_id_5 quantity_5
component_id_6 quantity_6
component_id_7 quantity_7
component_id_8 quantity_8
spec1
spec2
spec3
spec4
spec5
spec6
spec7
spec8
spec9
spec10


In [50]:
df_test_merged.head( 3 )

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,year,month,day,...,128_y,129_y,130_y,131_y,134_y,135_y,136_y,138_y,34_y.4,140_y
0,TA-00001,41,2013-06-23,0,0,1,1,2013,6,23,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,TA-00001,41,2013-06-23,0,0,1,2,2013,6,23,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,TA-00001,41,2013-06-23,0,0,1,5,2013,6,23,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


##4.10 Test : 불필요 Featrue 제거

In [51]:
df_test_merged = executeFeatureRemoval( df_test_merged )    # feature removal 처리를 수행
df_test_merged.head( 3 )

,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,year,month,day,material_id,diameter,...,128_y,129_y,130_y,131_y,134_y,135_y,136_y,138_y,34_y.4,140_y
0,41,0,0,1,1,2013,6,23,9,12.7,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,41,0,0,1,2,2013,6,23,9,12.7,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,41,0,0,1,5,2013,6,23,9,12.7,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


.

##4.11 Prediction Modeling

In [52]:
df_train_merged.head( 3 )

,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,cost,year,month,day,material_id,...,128_y,129_y,130_y,131_y,134_y,135_y,136_y,138_y,34_y.4,140_y
0,41,0,0,1,1,21.905933,2013,7,7,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,41,0,0,1,2,12.341214,2013,7,7,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,41,0,0,1,5,6.601826,2013,7,7,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [53]:
X = df_train_merged.drop( 'cost', axis = 1, inplace = False )    # X를 확보
y = np.log1p( df_train_merged[ 'cost' ] )   # y를 확보

df_train = ''
df_train_merged = ''

In [54]:
X.head( 3 )

,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,year,month,day,material_id,diameter,...,128_y,129_y,130_y,131_y,134_y,135_y,136_y,138_y,34_y.4,140_y
0,41,0,0,1,1,2013,7,7,1,6.35,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,41,0,0,1,2,2013,7,7,1,6.35,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,41,0,0,1,5,2013,7,7,1,6.35,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [55]:
y.head( 3 )

0    3.131396
1    2.590858
2    2.028389
Name: cost, dtype: float64

In [56]:
cv_cnt = 10    # cv : cross validation 횟수
n_jobs_cnt = 6

In [57]:
from sklearn.linear_model import SGDRegressor

from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import GradientBoostingRegressor

from sklearn import grid_search

In [58]:
if mode == True :
    model_list = [
                   RandomForestRegressor( max_depth = 100, n_estimators = 200, n_jobs = n_jobs_cnt ),    # 제출용
#                    RandomForestRegressor( n_jobs = n_jobs_cnt ),    # 제출용        
#                    RandomForestRegressor( n_estimators=5, n_jobs = n_jobs_cnt ),    # 제출용                
                 ]
else :
    model_list = [
                   RandomForestRegressor( n_jobs = n_jobs_cnt ),    # 테스트용
#                    GradientBoostingRegressor()
                 ]    

In [55]:
# X.to_csv( './dataset/xxxxxxxx.csv', index = False )
# X[ 'component_id_x.1' ]

In [56]:
# X.columns.tolist()

In [59]:
%%time
import time
for model in model_list :
    params = {    # 현재, 최적의 params 조건은 max_depth = 100, n_estimators = 500임
#                'max_depth' : (10,30,50,100,200),    # RandomForest
#                'n_estimators' : (10,20,50,100,200,500),    # RandomForest
             }

    gs = grid_search.GridSearchCV( model,
                                   param_grid = params,
#                                    n_jobs = n_jobs_cnt,
                                   cv = cv_cnt,
#                                    scoring = scorer
                                 )
    gs.fit( X, y )
    
    
    print( 'model : ', str( model ).split( '(' )[0] )
    print( 'best_score : ', gs.best_score_ )
    
    print( '=================' )
    print( 'best model : ', gs.best_estimator_ )
    print( '=================' )
    
    df_feature_importance = pd.DataFrame( X.columns.values, columns = [ 'features' ] )
    df_feature_importance[ 'importance' ] = gs.best_estimator_.feature_importances_
#     print( df_feature_importance.sort( 'importance', ascending = False  ) )    
    
    
    y_pred = gs.best_estimator_.predict( df_test_merged )    # prediction 수행
#     df_result[ 'cost' ] = y_pred
    df_result[ 'cost' ] = np.expm1( y_pred )
#     df_result[ 'cost' ] = df_result[ 'cost' ]
    
    now = time.strftime( '%Y%m%d%H%M%S' )    # 현재시각을 확보
    model_name = str( model ).split( '(' )[0]    # 파일생성용
    file_timestamp = now[2:4] + now[4:6] + now[6:8] + now[8:10] + now[10:12] + now[12:14]
    accuracy = '{0:.1f}%'.format( gs.best_score_ * 100 )    # latitude에 대한 예측률 저장 : 파일명 활용용도임
    df_result.to_csv( path_or_buf = './result.to.submit/' + file_timestamp + '.' + model_name +
                      '.' + accuracy + '.result().csv', sep = ',', index = False )

model :  RandomForestRegressor
best_score :  0.858809912793
best model :  RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=6, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
Wall time: 6min 5s


In [60]:
list_features = df_test_merged.columns.values.tolist()

In [61]:
with open( 'performance.condition.history.txt', 'a' ) as history_file :    # 모델의 이력을 logging
    history_file.write( '======================================================================\n' )
    history_file.write( '0. Description :\n' )
    if mode == True :
        history_file.write( '\tmode : Production mode\n' )
    else :
        history_file.write( '\tmode : Test mode\n' )
    history_file.write( '1. File name :\n\t' + file_timestamp + '.' + model_name + '.' + accuracy + '\n' )
    history_file.write( '2. Model information :\n\t' + str( gs.best_estimator_ ) + '\n' )
    history_file.write( '3. Applied features :\n' )
    for feature in list_features :
        history_file.write( '\t' + feature + '\n' )
    history_file.write( '4. Removed features :\n' )
    for feature in list_for_remove :
        history_file.write( '\t' + feature + '\n' )
    history_file.write( '5. Feature importance :\n\t' )        
    history_file.write( df_feature_importance.sort( 'importance', ascending = False  ).to_string() )
    history_file.write( '\n' )
    history_file.close()

In [62]:
df_result.head( 3 )

,id,cost
0,1,22.632436
1,2,13.271888
2,3,7.387373


In [63]:
df_result_desc = df_result.describe()    # 결과의 overview 확인
df_result_desc.drop( 'id', axis = 1, inplace = True )
df_result_desc.describe()

,cost
count,8.000000
mean,3874.356088
std,10654.077552
min,0.826364
25%,5.952737
50%,12.550039
75%,190.821622
max,30235.000000


In [64]:
X = ''    # X 초기화
y = ''

df_train = ''
df_train_merged = ''

df_test = ''
df_test_merged = ''

df_components = ''
df_merged = ''

df_result_desc = ''
df_result = ''
df_tmp = ''
df_feature_importance = ''

df_tube_bill = ''